![cao](img/cao.png)

<br>

# CAO Points Analysis

[Official CAO Website](https://www.cao.ie/)

***

## Overview
***

This notebook analyses the CAO points from the last three years by converting the following: 


- 2021 points from http into a dataframe.

- 2020 points from an excel spreadsheet into another dataframe.

- 2019 points from a pdf into a dataframe. 



Then I combine all three points into one dataframe for analysis. 

<br>

## Importing modules
***

### Regular Expressions
Regular expression also known as [regexes](https://realpython.com/regex-python/) are special sequence of characters that are used to form a search pattern. In other words, a user can use a regular expression to search through a particular file in order to find that particular pattern or sequence. Python has a built-package for regular expressions called "[re](https://docs.python.org/3/library/re.html)". 


### Requests
This is another built-in package in python. [Requests](https://www.pythonforbeginners.com/requests/using-requests-in-python) is imported to allow a user to send HTTP/1.1 requests. To put it simply, this module contains various functions and operations that allow the user to retrieve data from a http. 


### Datetime
The [datetime module](https://www.geeksforgeeks.org/python-datetime-module/) is imported when working with dates and times. There are six main categories in this module:

1. date - used for year, month or day.


2. time - used for hours, minutes, seconds, microseconds, and tzinfo.


3. datetime - is a combination of both date and time. 


4. timedelta - used to represent duration.


5. tzinfo - gives time zone information objects.


6. timezone – gives tzinfo as fixed offset from UTC. 


### Pandas
[Pandas](https://mode.com/python-tutorial/libraries/pandas/#:~:text=Pandas%20is%20a%20Python%20library%20for%20data%20analysis.&text=Pandas%20is%20built%20on%20top,NumPy%27s%20methods%20with%20less%20code.) is another python library used for data analysis. It uses dataframes and operations to manipulate numerical tables and time series. Pandas will be used in this notebook to store and compare the cao points from 2021, 2020 and 2019. 


### Urllib request
This is an [extensive library](https://docs.python.org/3/library/urllib.request.html) used for opening urls. Unlike the requests library, this module offers more functionality and quicker way to open and read data from urls. 

<br>

In [1]:
# For regular expressions. - ref 1.
import re

# Convenient HTTP requests - ref 4.
import requests as rq

# Dates and times. - ref 8. 
import datetime as dt

# Pandas for data frames. - ref 9.
import pandas as pd

# For downloading urls. - ref 10.
import urllib.request as urlrq

# references to use at end!!!!!


- https://realpython.com/regex-python/


- https://realpython.com/regex-python-part-2/


- https://www.mygreatlearning.com/blog/regular-expression-in-python/


- https://docs.python-requests.org/en/latest/user/quickstart/#make-a-request


- https://howchoo.com/g/ywi5m2vkodk/working-with-datetime-objects-and-timezones-in-python


- https://pandas.pydata.org/pandas-docs/stable/user_guide/dsintro.html


- https://stackoverflow.com/questions/16870648/python-read-website-data-line-by-line-when-available


- https://www.geeksforgeeks.org/python-urllib-module/

<br>

## Time stamp
***

Here I created a time stamp by getting the current date and time using the datetime module. Next the strftime function is used to take in the current date and time and convert it into a string using the strftime method. There are numerious formats that can be used. A list of the strftime format can be found [here](https://strftime.org/). 


In the code cell below, I take the current date and time and convert it into the following format. This will then be used as the time stamp going forward.

- %Y stands for the year as a decimal number.


- %m represents the month as a decimal number.


- %d is for the day of the month as a decimal number.


- %H gives the hour in a 24 hour clock format.


- %M is used to get the minute as a decimal number.


- %S produces the second as a decimal number.


In [2]:
# Get the current date and time.
now = dt.datetime.now()

# Format as a string.
nowstr = now.strftime('%Y%m%d_%H%M%S')

<br>

# 2021 CAO Points

http://www.cao.ie/index.php?page=points&p=2021
***

This section converts the 2021 CAO points from a http link into a pandas dataframe. 

<br>

### Request the http link
***

In the code cell below, [requests.get](https://docs.python-requests.org/en/latest/user/quickstart/#make-a-request) is used to fetch the data from the selected http link. 


It is also good practice to check the http respone. This is done by call `resp`. As you can see the returned respone is 200 which means the request was successful. A list of different response codes can be found [here](https://developer.mozilla.org/en-US/docs/Web/HTTP/Status)


In [3]:
# Fetch the 2021 CAO points URL - ref 4.
resp = rq.get('http://www2.cao.ie/points/l8.php')

# Check response. 200 means OK. 401 means error. 
resp

<Response [200]>

<br>

### Save origional data set
***

Here I use the time stamp I created above to open a file path for the origional http data. It will be stored in the data folder in this repository and the time stamp is used in the filename in order to keep track of the data each time the code is run in case an error ever occurs. 


In [4]:
# Create a file path for the original data.
pathhtml = 'data/cao2021_' + nowstr + '.html'

<br>

### Error on server
***

[Encoding](https://stackoverflow.com/questions/4657416/difference-between-encoding-and-encryption) is used to transfer data in a safe way so that it can be used on different systems. The encoding on my machine may differ from anothers which is why we need to decode using the method of that particular server.


The server says we should decode as per:

```Content-Type: text/html; charset=iso-8859-1```


However, an error occured. One of the lines uses \x96 which isn't defined in iso-8859-1. This error was referring to a fada in one of the Irish course names. 


Therefore, we need to use a similar decoding standard [cp1252](https://en.wikipedia.org/wiki/Windows-1252#:~:text=Windows%2D1252%20or%20CP%2D1252,Spanish%2C%20French%2C%20and%20German.) instead. This encoding method includes characters from the Latin alphabet which would include fadas. 

In [5]:
# The server uses the wrong encoding.
original_encoding = resp.encoding

# Change encoding here. 
resp.encoding = 'cp1252'

<br>

Using a [with statement](https://www.pythonforbeginners.com/files/with-statement-in-python) here to open file created above and write the above request to that file. 

In [6]:
# Open and write origional http data to file.
with open(pathhtml, 'w') as f:
    f.write(resp.text)

<br>

### Regular Expressions
***

Using a regular expression here to select the desired lines within the http file.  This [blog](https://www.mygreatlearning.com/blog/regular-expression-in-python/) gives very clear explaination of characters used when working with regular expressions. In the code cell below:


- [re.compile](https://www.tutorialspoint.com/Why-do-we-use-re-compile-method-in-Python-regular-expression) collects a regular expression pattern into pattern objects. 


- The [r](https://developers.google.com/edu/python/regular-expressions#:~:text=The%20%27r%27%20at%20the%20start,needs%20this%20feature%20badly!) at the start of the regular expression below refers to the start of the pattern string. 


- The letters or numbers inside the square brackets set what you are searching for. 


- The number within the braces set the amount of characters to search for.  


- An important note here is the use of two blank spaces as part of this regualr expression. If these characters were not added then the expression would not work correctly.


- Finally, the full stop look for any character, except for a new line and the asterisk looks to match the preceding characters zero or more times.  


In [7]:
# Compile the regular expression for matching lines.
re_course = re.compile(r'([A-Z]{2}[0-9]{3})  (.*)')

<br>

### Loop through the lines of the response
***

This funciton is used to seperate the points from the # and * characters and then store them in an array. 

- First the variable portfolio is created for the # characters. The origional http file told us that # represents if there was a entry test, interview, portfolio or audition for the course. An [if statement](https://www.w3schools.com/python/python_conditions.asp) is used in this function to see if any value is equal to the # symbol and then stores it in the portofio variable. 


- Next the random variable checks for * characters. This informs us that not all on this points score were offered places.


- Finally the [isdigit](https://www.w3schools.com/python/ref_string_isdigit.asp) function is used to check if any of the characters in the text are digits and stores them in the points variable. 


In [8]:
# Creating a function to seperate and add points # and * to an array.
def points_to_array(s):
    portfolio = ''
    if s[0] == '#':
        portfolio = '#'
        
    random = ''
    if s[-1] == '*':
        random = '*'
        
    points = ''
    
    for i in s:
        if i.isdigit():
            # not addition but linking these two things together 
            points = points + i 
            
    return[points, portfolio, random]


In [9]:
# Creating a path for new 2021 points as a csv file.
path2021 = 'data/cao2021_csv_' + nowstr + '.csv' 

In [10]:
<br>

In the code cell below:
    

SyntaxError: invalid syntax (<ipython-input-10-8f52afea4bea>, line 1)

<br>

In the code cell below:

- [iter_lines](https://stackoverflow.com/questions/16870648/python-read-website-data-line-by-line-when-available) is a function in the requests library which reads the data line by line. It is better to take small bits of data at a time in to avoid losing anything. 


- [fullmatch](https://www.geeksforgeeks.org/re-fullmatch-function-in-python/) returns a match object if the whole string matches the regular expression pattern. 


- [strip](https://www.w3schools.com/python/ref_string_strip.asp) is used to remove spaces or characters either a the beginning or the end of a string. So far the first five values are assigned to the course_code variable. Next the strip function is used to extract characters from the seventh value up until the fifty seventh value and this is set to the course_title. 


- [re.split](https://stackoverflow.com/questions/12683201/python-re-split-to-split-by-spaces-commas-and-periods-but-not-in-cases-like) returns a list where the string is split at each match. In other words, the points are split using the regular expression and the plus symbol will match one or more of that pattern. So this splits the points into round 1 and round 2. 


- [len](https://www.w3schools.com/python/ref_func_len.asp) is used to check if the length of the course list is not equal to two then only return first two values. 

<br>

Now the amended 2021 points with only the information I want is written to a new csv file that can be used for analysis.

In [ ]:
# Keeps track of how many courses we process.
no_lines = 0

# Open the csv file for writing.
with open(path2021, 'w') as f:
    # Write a header row for columns.
    f.write(','.join(['code', 'title', 'pointsR1', 'pointsR2']) + '\n')
    
    # Loop through lines of the response - ref 6.
    for line in resp.iter_lines():
        
        # Decode the line, using the wrong encoding!
        dline = line.decode('cp1252')
        
        # Match only the lines representing courses.
        if re_course.fullmatch(dline):
            
            # Add one to the lines counter.
            no_lines = no_lines + 1
            
            #extract first 5 characters for course codes.
            course_code = dline[:5]
            
            # strip everything before 7  and after 57
            course_title = dline[7:57].strip()
            
            #course_round1 = dline[60:]
            #print(f"'{course_code} {len(dline)}'")
            
            course_points = re.split(' +', dline[60:])
            #print(f"'{course_code} {course_points}'")
            
            # if the length of points is not equal to 2 only return first 2 values. 
            if len(course_points) != 2:
                
                # prints out last line which contains 3 
                # print(f"'{course_code} {course_points}'")
                
                course_points = course_points[:2]
                
            # join the fields using a comma.
            linesplit = [course_code, course_title, course_points[0], course_points[1]]
            
            # Rejoin the substrings with commas in between.
            f.write(','.join(linesplit) + '\n')
               
# Print the total number of processed lines.
print(f"Total number of lines is {no_lines}.")


# TO DO HERE

- tidy up 2021 points 


- write a sub function to deal with the * and # components to seperate from points


- more comments and explainations

<br>

# VERIFY THIS MANUALLY!!!! 


**NB**: It was verified as of //21 that there were 949 courses exactly on the CAO 2021 points list

***

In [ ]:
df2021 = pd.read_csv(path2021, encoding='cp1252')

In [ ]:
df2021

***

<br>

# 2020 CAO Points

http://www.cao.ie/index.php?page=points&p=2020

***

In [ ]:
url2020 = 'http://www2.cao.ie/points/CAOPointsCharts2020.xlsx'

<br>

### Save origional data set

***

In [ ]:
# Create a file path for the original data.
pathxlsx = 'data/cao2020_' + nowstr + '.xlsx'

In [ ]:
urlrq.urlretrieve(url2020, pathxlsx)

<br>

### Load Excel Spreadsheet using Pandas

***

In [ ]:
# Download and parse the excel spreadsheet.
df2020 = pd.read_excel(url2020, skiprows=10)

In [ ]:
df2020

In [ ]:
# Spot check a random row.
df2020.iloc[753]

In [ ]:
# Spot check the last row.
df2020.iloc[-1]

In [ ]:
# Create a file path for the pandas data.
path2020 = 'data/cao2020_' + nowstr + '.csv'

In [ ]:
# Save pandas data frame to disk.
df2020.to_csv(path2020)

 <br>

## 2019 CAO Points

http://www.cao.ie/index.php?page=points&p=2019

***   

**Steps to reproduce**

1.  Download original pdf file.
2.  Open original pdf file in Microsoft Word.
3.  Save Microsoft Word's converted PDF in docx format.
4.  Re-save Word document for editing.
5.  Delete headers and footers.
6.  Delete preamble on page 1.
7.  Select all and copy.
8.  Paste into Visual Studio Code.
9.  Remove HEI name lines and black lines.
10. Change column heading "COURSE AND INSTITUTION" to "Course".
11. Change backticks to apostrophes.
12. Replaced double tab charater at on line 28 with single tab.
13. Delete tabs at end of lines 604, 670, 700, 701, 793, and 830.

In [ ]:
df2019 = pd.read_csv('data/cao2019_20211103_202410_edited.csv', sep='\t')

In [ ]:
df2019

EOS means end of season points. 

MID means mid points person had who got the course

***

# Concat and join
***

In [ ]:
# CREATED DF CALLED COURSES2021

In [ ]:
courses2021 = df2021[['code', 'title']]
courses2021

In [ ]:
# CREATED DF CALLED COURSES2020

In [ ]:
courses2020 = df2020[['COURSE CODE2','COURSE TITLE']]
# set column heading to be the same as 2021
courses2020.columns = ['code', 'title']
courses2020

In [ ]:
# CONCATENATE COURSES2021 AND COURSES2020 
# PUT CODE AND TITLE ON TOP OF ONE ANOTHER

In [ ]:
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.concat.html

allcourses = pd.concat([courses2021, courses2020], ignore_index=True)
allcourses

In [ ]:
# SORT THE VALUES TO SHOW THERE ARE DUPLICATES IN THE CONCATENATED DF

In [ ]:
allcourses.sort_values('code')

In [ ]:
# DISPLAY DUPLICATE COURSE ON TWO INDEX'

In [ ]:
allcourses.loc[175]['title']

In [ ]:
allcourses.loc[949]['title']

In [ ]:
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.duplicated.html

# Finds all extra copies of duplicated rows.
allcourses[allcourses.duplicated()]

In [ ]:
# Returns a copy of the data frame with duplciates removed.
allcourses.drop_duplicates()

In [ ]:
# REMOVE DUPLICATES BASED ON CODE ALONE

In [ ]:
# Finds all extra copies of duplicated rows.
allcourses[allcourses.duplicated(subset=['code'])]

In [ ]:
# DF OF ALLCOURSES
# CONTAINS FULL LIST OF COURSES AVAILABLE IN 2021, 2020 OR BOTH

In [ ]:
# INPLACE=TRUE MEANS IT MAKES THE CHANGE IN THE DF AS OPPOSED TO RETURNING A NEW ONE

#IGNORE_INDEX=TRUE IGNORES THE INDES OF ORIGIONAL ARRAYS AND BASICALLY RE-SETS THE INDEX ON THE NEW DF

In [ ]:
# Returns a copy of the data frame with duplciates removed - based only on code.
allcourses.drop_duplicates(subset=['code'], inplace=True, ignore_index=True)

In [ ]:
allcourses

<br>

# Join to points

In [ ]:
# INPLACE=TRUE AGAIN PERMANENTLY CHANGES THE INDEX OF DF2021 AND SETS IT AS THE CODE

In [ ]:
# Set the index to the code column.
df2021.set_index('code', inplace=True)
df2021.columns = ['title', 'points_r1_2021', 'points_r2_2021']
df2021

In [ ]:
# INPLACE=TRUE AGAIN PERMANENTLY CHANGES THE INDEX OF ALLCOURSES AND SETS IT AS THE CODE

In [ ]:
# Set the index to the code column.
allcourses.set_index('code', inplace=True)

In [ ]:
# NOW JOINING POINTS FROM DF2021 ARRAY TO ALLCOURSES ARRAY

In [ ]:
allcourses = allcourses.join(df2021[['points_r1_2021']])
allcourses

In [ ]:
# DOING THE SAME THING FOR DF2020

In [ ]:
df2020_r1 = df2020[['COURSE CODE2', 'R1 POINTS']]
df2020_r1.columns = ['code', 'points_r1_2020']
df2020_r1

In [ ]:
# PERMANETLY CHANGING INDEX OF DF2020 TO THE CODE COLUMN

In [ ]:
# Set the index to the code column.
df2020_r1.set_index('code', inplace=True)
df2020_r1

In [ ]:
# FINALLY JOINING DF2020 TO ALL COURSES DF
# CREATES A TABLE CONTAIN COLUMN FOR 2021 POINTS AND 2020 PONITS
# https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.join.html

In [ ]:
# Join 2020 points to allcourses.
allcourses = allcourses.join(df2020_r1)
allcourses

# TO DO


***

- ADD EXPLANATIONS ABOUT WHAT IS HAPPENING


- MAYBE ADD PLOTS TO COMPARE DATA


- ADD CONTENT TO THE README


- SAVE NEW DF OF ALLCOURSES TO A CSV IN DATA FOLDER


- JOIN IN 2019 POINTS TO ALLCOURSES DF AND COMPARE


<br>


# This notebook should have:
 ***
 
 1. origional data file 2021,2020,2019 from cao website
 
 
2. cleaned data files x 3 again


3. merged data file with all 3 - analyse this one with plots

<br>

# References:

***
 
All references and code used in these notebooks have been sourced in Oct/Nov/Dec 2021 from the following webpages:

 
- https://docs.python.org/3/library/re.html


- https://realpython.com/regex-python/


- https://realpython.com/regex-python-part-2/


- https://docs.python-requests.org/en/latest/user/quickstart/#make-a-request


- https://www.mygreatlearning.com/blog/regular-expression-in-python/


- https://stackoverflow.com/questions/16870648/python-read-website-data-line-by-line-when-available


- https://sites.pitt.edu/~naraehan/python3/mbb12.html


- https://docs.python.org/3/library/datetime.html


- https://pandas.pydata.org/pandas-docs/stable/user_guide/dsintro.html


- https://docs.python.org/3/library/urllib.request.html


- 




<br>

***

# End